In [1]:
# Run inference from a sample image 

# image_filepath = "../images_data/test_images/noisy_image.jpg" # Noisy Test Image
image_filepath = "../images_data/dslr_night_images_gray/focused_star_dsc0014_752x502_gray.png" # Night Sky Image
# image_filepath = "../images_data/mag5_1608_47deg_gray/122001.png"
# image_filepath = "../images_data/mag5_1608_47deg_gray/11345001.png" # Star with least stars in frame (106 other stars)
model_filepath = "../ml/models/mag5_1608_47deg_64_128_100epoch.keras"

bin_count = 10
bin_max_radius = 400

import os
import sys
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image
from photutils.centroids import centroid_sources
from astropy.stats import sigma_clipped_stats
from astropy.io import fits
from photutils.detection import DAOStarFinder
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture

sys.path.append(os.path.join(os.getcwd(), '..', 'algorithms'))
import centroid as ct
import feature_extraction as bin

2024-07-07 14:27:21.884653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 14:27:21.978403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 14:27:21.978888: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 14:27:22.081925: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 14:27:23.085556: W tensorflow/compiler/tf

In [2]:

np.set_printoptions(threshold=sys.maxsize)

centroids = []
bins = []
img = Image.open(image_filepath)

# Calculate centroids for each image
centroids = ct.centroids_from_img(img)
centroids = centroids['xcentroid', 'ycentroid']
# print("Centroids: ", centroids)

# Binning Feature Extraction
bins = bin.feature_extraction_binning(centroids, 752, 480, bin_max_radius, bin_count)

# loaded_model = tf.saved_model.load(model_filepath)
loaded_model = tf.keras.models.load_model(model_filepath)
print("\nLoaded Model\n")

bins = np.array(bins)
reshaped_bins = bins.reshape(-1,10)
print("Reshaped Bins: ", reshaped_bins)

predictions = loaded_model(reshaped_bins)
# print("Predictions: ", predictions)
print("Prediction Confidence: ", np.max(predictions),"\n") 

# sorted_predictions = np.sort(predictions) 
# print("Top 3 Predictions: ", sorted_predictions[-3:])

# TODO: Figure out labels


temp_y_label_source = "../images_data/mag5_1608_47deg_gray/bins/mag5_1608_47deg_bin_features.csv"
# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(temp_y_label_source, header=0, names=["HIP", "bin0", "bin1", "bin2", "bin3", "bin4", "bin5", "bin6", "bin7", "bin8", "bin9"])
dataset = dataframe.values
Y = dataset[:,0].astype(int)
encoder = LabelEncoder()
encoder.fit(Y)

print("Prediction Index: ", predictions.numpy().argmax())
print("Label: ", encoder.classes_[predictions.numpy().argmax()])

# encoded_predictions = np.zeros(len(predictions.numpy().flatten())).astype(int)
# encoded_predictions[predictions.numpy().argmax()] = 1
# print("Encoded Predictions: ",encoded_predictions)

# label = encoder.inverse_transform(encoded_predictions)
# print("Label: ", label)

FileNotFoundError: [Errno 2] No such file or directory: '/home/mark/GitHub/star-tracker-ml/images_data/dslr_night_images_gray/focused_star_dsc0014_752x502_gray.png'